<a href="https://colab.research.google.com/github/cod3astro/dsnBootcamp/blob/main/colab_dsn_bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category-encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from itertools import combinations
from scipy.stats import randint, uniform

In [3]:
df = pd.read_csv('train.csv', index_col='id')
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
df.shape

(188533, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 18.7+ MB


In [6]:
df.describe()

,model_year,milage,price
count,188533.000000,188533.000000,1.885330e+05
mean,2015.829998,65705.295174,4.387802e+04
std,5.660967,49798.158076,7.881952e+04
min,1974.000000,100.000000,2.000000e+03
25%,2013.000000,24115.000000,1.700000e+04
50%,2017.000000,57785.000000,3.082500e+04
75%,2020.000000,95400.000000,4.990000e+04
max,2024.000000,405000.000000,2.954083e+06


In [7]:
df.describe(include='object')

,brand,model,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
count,188533,188533,183450,188533,188533,188533,188533,186081,167114
unique,57,1897,7,1117,52,319,156,2,1
top,Ford,F-150 XLT,Gasoline,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes
freq,23088,2945,165940,3462,49904,48658,107674,144514,167114


In [8]:
df.isnull().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0
accident,2452


In [9]:
for col in df.columns:
  unique_columns = df[col].unique()
  print(f'{col}: has {len(unique_columns)} unique values')
  print(unique_columns)

brand: has 57 unique values
['MINI' 'Lincoln' 'Chevrolet' 'Genesis' 'Mercedes-Benz' 'Audi' 'Ford'
 'BMW' 'Tesla' 'Cadillac' 'Land' 'GMC' 'Toyota' 'Hyundai' 'Volvo'
 'Volkswagen' 'Buick' 'Rivian' 'RAM' 'Hummer' 'Alfa' 'INFINITI' 'Jeep'
 'Porsche' 'McLaren' 'Honda' 'Lexus' 'Dodge' 'Nissan' 'Jaguar' 'Acura'
 'Kia' 'Mitsubishi' 'Rolls-Royce' 'Maserati' 'Pontiac' 'Saturn' 'Bentley'
 'Mazda' 'Subaru' 'Ferrari' 'Aston' 'Lamborghini' 'Chrysler' 'Lucid'
 'Lotus' 'Scion' 'smart' 'Karma' 'Plymouth' 'Suzuki' 'FIAT' 'Saab'
 'Bugatti' 'Mercury' 'Polestar' 'Maybach']
model: has 1897 unique values
['Cooper S Base' 'LS V8' 'Silverado 2500 LT' ... 'e-Golf SE'
 'Integra w/A-Spec Tech Package' 'IONIQ Plug-In Hybrid SEL']
model_year: has 34 unique values
[2007 2002 2017 2021 2018 2016 2020 2015 2011 2013 2023 2019 2012 2014
 2008 2009 2022 2003 2005 2001 2006 2000 2010 2004 1997 1998 1999 1994
 1993 1996 1995 2024 1974 1992]
milage: has 6651 unique values
[213000 143250 136731 ...   4721   2922 134603]
fue

In [10]:
drop_df = df.copy()
drop_df = drop_df.dropna()
drop_df.shape

(162610, 12)

In [11]:
missing_columns = df.columns[df.isnull().any()].to_list()
for col in missing_columns:
    df[f"{col}_missing"] = df[col].isnull().astype(int)
df.fillna(df.mode().iloc[0], inplace=True)
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,fuel_type_missing,accident_missing,clean_title_missing
id,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,0,0,0
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,0,0,0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,0,0,0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,0,0,0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,0,0,0


In [12]:
df.isnull().any().any()

np.False_

In [13]:
df.shape

(188533, 15)

In [14]:
new_features = {}
data = df.copy()
cat_cols = data.select_dtypes(include=['object', 'category']).columns
for r in [2, 3, 4]:
    for cols in combinations(cat_cols, r):
        new_col_name = "_".join(cols)
        new_features[new_col_name] = data[list(cols)].astype(str).agg("_".join, axis=1)

new_features_data = pd.DataFrame(new_features)
data = pd.concat([data, new_features_data], axis=1)
data.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,engine_transmission_accident_clean_title,engine_ext_col_int_col_accident,engine_ext_col_int_col_clean_title,engine_ext_col_accident_clean_title,engine_int_col_accident_clean_title,transmission_ext_col_int_col_accident,transmission_ext_col_int_col_clean_title,transmission_ext_col_accident_clean_title,transmission_int_col_accident_clean_title,ext_col_int_col_accident_clean_title
id,,,,,,,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,...,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel_A...,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel_Y...,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel_Y...,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel_Y...,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel_G...,A/T_Yellow_Gray_None reported,A/T_Yellow_Gray_Yes,A/T_Yellow_None reported_Yes,A/T_Gray_None reported_Yes,Yellow_Gray_None reported_Yes
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,...,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel_A...,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel_S...,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel_S...,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel_S...,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel_B...,A/T_Silver_Beige_At least 1 accident or damage...,A/T_Silver_Beige_Yes,A/T_Silver_At least 1 accident or damage repor...,A/T_Beige_At least 1 accident or damage report...,Silver_Beige_At least 1 accident or damage rep...
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,...,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T_Blue_Gray_None reported,A/T_Blue_Gray_Yes,A/T_Blue_None reported_Yes,A/T_Gray_None reported_Yes,Blue_Gray_None reported_Yes
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,...,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel_T...,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel_B...,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel_B...,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel_B...,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel_B...,Transmission w/Dual Shift Mode_Black_Black_Non...,Transmission w/Dual Shift Mode_Black_Black_Yes,Transmission w/Dual Shift Mode_Black_None repo...,Transmission w/Dual Shift Mode_Black_None repo...,Black_Black_None reported_Yes
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,...,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel_7...,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel_B...,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel_B...,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel_B...,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel_B...,7-Speed A/T_Black_Beige_None reported,7-Speed A/T_Black_Beige_Yes,7-Speed A/T_Black_None reported_Yes,7-Speed A/T_Beige_None reported_Yes,Black_Beige_None reported_Yes


In [15]:
data.shape

(188533, 261)

In [16]:
data.isnull().any().any()

np.False_

In [17]:
print(df.shape)
print(drop_df.shape)
print(data.shape)

(188533, 15)
(162610, 12)
(188533, 261)


In [18]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix, roc_auc_score
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from scipy.stats import pointbiserialr, chi2_contingency
from imblearn.pipeline import Pipeline as ImbPipeline

In [19]:
target = 'price'
features = df.drop(columns=[target]).columns
drop_features = drop_df.drop(columns=[target]).columns
data_features = data.drop(columns=[target]).columns

print(df[target].skew())
print(drop_df[target].skew())
print(data[target].skew())

df["target_log_df"] = np.log1p(df[target])
drop_df["target_log_drop"] = np.log1p(drop_df[target])
data["target_log_data"] = np.log1p(data[target])
print(df["target_log_df"].skew())
print(drop_df["target_log_drop"].skew())
print(data["target_log_data"].skew())

X, y = df[features].reset_index(drop=True), df['target_log_df'].reset_index(drop=True)
A, b = drop_df[drop_features], drop_df['target_log_drop']
C, d = data[data_features], data['target_log_data']
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
train_A, val_A, train_b, val_b = train_test_split(A, b, random_state=1)
train_C, val_C, train_d, val_d = train_test_split(C, d, random_state=1)

20.268452710951134
21.424949839413017
20.268452710951134
0.10786368406837661
0.14902187532889827
0.10786368406837661


In [21]:
print("First 5 indices of X:", X.index[:5])
print("First 5 indices of y:", y.index[:5])
print("Do they match?", X.index.equals(y.index))

First 5 indices of X: RangeIndex(start=0, stop=5, step=1)
First 5 indices of y: RangeIndex(start=0, stop=5, step=1)
Do they match? True


In [20]:
print("X index length:", len(X.index))
print("y index length:", len(y.index))
print("Unique indices in X not in y:", X.index.difference(y.index)[:10])
print("Unique indices in y not in X:", y.index.difference(X.index)[:10])

X index length: 188533
y index length: 188533
Unique indices in X not in y: RangeIndex(start=0, stop=0, step=1)
Unique indices in y not in X: RangeIndex(start=0, stop=0, step=1)


In [22]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Length of df:", len(df))
print("Any NaN in target_log_df?", df['target_log_df'].isnull().sum())

Shape of X: (188533, 14)
Shape of y: (188533,)
Length of df: 188533
Any NaN in target_log_df? 0


In [23]:
target = 'price'
numerical_col = ['model_year', 'milage']
exclude_cols = numerical_col + [target, "target_log_df"]
categorical_col = df.drop(columns=exclude_cols).columns.tolist()

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1)) / (n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

X_num = df[numerical_col]
y = df['target_log_df']
mi_scores = mutual_info_regression(X_num, y, discrete_features=False)
mi_scores_dict = dict(zip(numerical_col, mi_scores))

target_binned = pd.qcut(df[target], q=10)
cramers_scores = {col: cramers_v(df[col], target_binned) for col in categorical_col}

all_scores = {**cramers_scores, **mi_scores_dict}
all_scores_series = pd.Series(all_scores).sort_values(ascending=False)

print("\n📊 Combined Feature Correlation Report with Target:")
print(all_scores_series)

/tmp/ipython-input-193984993.py:15: RuntimeWarning: invalid value encountered in scalar divide
  return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))



📊 Combined Feature Correlation Report with Target:
milage                 0.657380
model_year             0.543202
engine                 0.315676
accident               0.313321
model                  0.298762
clean_title_missing    0.223095
transmission           0.196733
brand                  0.145614
int_col                0.128732
ext_col                0.127367
fuel_type_missing      0.118881
fuel_type              0.061920
accident_missing       0.033364
clean_title                 NaN
dtype: float64


In [24]:
target = 'price'
numerical_drop_col = ['model_year', 'milage']
exclude_drop_col = numerical_drop_col + [target, "target_log_drop"]
categorical_drop_col = drop_df.drop(columns=exclude_drop_col).columns.tolist()

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1)) / (n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

X_num_drop = drop_df[numerical_drop_col]
y_drop = drop_df['target_log_drop']
mi_drop_scores = mutual_info_regression(X_num_drop, y_drop, discrete_features=False)
mi_drop_scores_dict = dict(zip(numerical_drop_col, mi_drop_scores))

target_binned = pd.qcut(df[target], q=10)
cramers_drop_scores = {col: cramers_v(df[col], target_binned) for col in categorical_drop_col}

all_drop_scores = {**cramers_drop_scores, **mi_drop_scores_dict}
all_drop_scores_series = pd.Series(all_drop_scores).sort_values(ascending=False)

print("\n📊 Combined Feature Correlation Report with Target:")
print(all_drop_scores_series)


📊 Combined Feature Correlation Report with Target:
milage          0.638510
model_year      0.519708
engine          0.315676
accident        0.313321
model           0.298762
transmission    0.196733
brand           0.145614
int_col         0.128732
ext_col         0.127367
fuel_type       0.061920
clean_title          NaN
dtype: float64


/tmp/ipython-input-2882386070.py:15: RuntimeWarning: invalid value encountered in scalar divide
  return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", TargetEncoder(cols=categorical_col, smoothing=0.3), categorical_col),
        ("num", "passthrough", numerical_col)
    ]
)

xgb = XGBRegressor(objective="reg:squarederror", random_state=42, n_jobs=-1)

pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", xgb)
])

param_dist = {
    "model__n_estimators": randint(100, 1000),
    "model__max_depth": randint(3, 15),
    "model__learning_rate": uniform(0.01, 0.3),
    "model__subsample": uniform(0.5, 0.5),
    "model__colsample_bytree": uniform(0.5, 0.5),
    "model__min_child_weight": randint(1, 10),
    "model__gamma": uniform(0, 5)
}

random_search_xgb = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    random_state=42,
    verbose=2
)

random_search_xgb.fit(train_X, train_y)

print("Best Parameters:", random_search_xgb.best_params_)
print("Best RMSE:", -random_search_xgb.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'model__colsample_bytree': np.float64(0.8121770240668966), 'model__gamma': np.float64(1.47816842918857), 'model__learning_rate': np.float64(0.041648277949081186), 'model__max_depth': 14, 'model__min_child_weight': 4, 'model__n_estimators': 897, 'model__subsample': np.float64(0.9416401294594341)}
Best RMSE: 0.4962480535187437


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", TargetEncoder(cols=categorical_col, smoothing=0.3), categorical_col),
        ("num", "passthrough", numerical_col)
    ]
)
df_rfModel = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))
])
scores = cross_val_score(df_rfModel, X, y, cv=5, scoring="neg_root_mean_squared_error")
print("Average RMSE:", -scores.mean())

Average RMSE: 0.5087551879575528


In [26]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", TargetEncoder(cols=categorical_col), categorical_col),
        ("num", "passthrough", numerical_col)
    ]
)
df_XGBModel = Pipeline([
    ("preprocessor", preprocessor),
    ("model", XGBRegressor(n_estimators=897, random_state=42, max_depth=14, learning_rate=0.04, gamma=1.478, colsample_bytree=0.812,
                           min_child_weight=4, subsample=0.942))
])
scores = cross_val_score(df_XGBModel, X, y, cv=5, scoring="neg_root_mean_squared_error")
print("Average RMSE:", -scores.mean())

Average RMSE: 0.49365250322972576


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", TargetEncoder(cols=categorical_drop_col, smoothing=0.3), categorical_drop_col),
        ("num", "passthrough", numerical_drop_col)
    ]
)
drop_rfModel = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))
])
drop_scores = cross_val_score(drop_rfModel, A, b, cv=5, scoring="neg_root_mean_squared_error")
print("Average RMSE:", -drop_scores.mean())

Average RMSE: 0.5070781447382197


In [34]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_drop_col),
        ("num", 'passthrough', numerical_drop_col)
    ]
)
drop_xgbModel = Pipeline([
    ("preprocessor", preprocessor),
    ("model", XGBRegressor(n_estimators=897, random_state=42, max_depth=14, learning_rate=0.04, gamma=1.478, colsample_bytree=0.812,
                           min_child_weight=4, subsample=0.942))
])
drop_scores = cross_val_score(drop_xgbModel, A, b, cv=5, scoring="neg_root_mean_squared_error")
print("Average RMSE:", -drop_scores.mean())

Average RMSE: 0.49086187273679194


In [28]:
numerical_data_col = ['model_year', 'milage']
exclude_data_col = numerical_data_col + [target, "target_log_data"]
categorical_data_col = data.drop(columns=exclude_data_col).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", TargetEncoder(cols=categorical_data_col, smoothing=0.3), categorical_data_col),
        ("num", "passthrough", numerical_data_col)
    ]
)
data_xgbModel = Pipeline([
    ("preprocessor", preprocessor),
    ("model", XGBRegressor(n_estimators=897, random_state=42, max_depth=14, learning_rate=0.04, gamma=1.478, colsample_bytree=0.812,
                           min_child_weight=4, subsample=0.942))
])
# data_xgbModel.fit(train_C, train_d)
# data_xgb_preds = data_xgbModel.predict(val_C)
# rmse = mean_squared_error(val_d, data_xgb_preds)
# print("Validation RMSE:", rmse)
data_scores = cross_val_score(data_xgbModel, C, d, cv=5, scoring="neg_root_mean_squared_error")
print("Average RMSE:", -data_scores.mean())

Average RMSE: 0.4961890492305382
